<a href="https://colab.research.google.com/github/zack-dev-cm/mcp_server/blob/main/MCP_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running the MCP demo in Colab
This notebook starts the server and shows how to query it.

In [ ]:
!pip install fastapi uvicorn[standard] gradio==4.* httpx openai pydantic pydantic-settings python-dotenv --quiet

In [ ]:
from colab_adapter import launch_in_colab
launch_in_colab()

In [ ]:
import requests, time
time.sleep(2)
requests.get("http://localhost:8000/v1/resources").json()

Open the URL printed above to access the Gradio UI.